##part b) Neural network with pytorch with autodifferentiation

---



---






In [38]:
import numpy as np
import random
from numpy import array
from numpy.random import uniform
from numpy import hstack
import torch
import torch.nn as nn

In [39]:

n, d = 400, 3
num_hidden1=5 # number of neurons for first hidden layer
num_hidden2=4 # numbe rof neurons for second hidden layer
x = torch.FloatTensor(n, d).uniform_(-1, 1)
weights_true = torch.tensor([[5,1,1],[4,1,1]]).float()
weights_true = torch.transpose(weights_true,0,1)
bias_true = torch.tensor([1])
y_true = torch.mm(x**2,weights_true) + torch.mm(x,weights_true) + bias_true
print(f'x: {x.shape}, weights: {weights_true.shape}, bias: {bias_true.shape}, y: {y_true.shape}')

'''
# Generating data of 3 dim input and 2 dim output
n, d = 500, 3
x = torch.FloatTensor(n, d).uniform_(-1, 1)
weights_true = torch.tensor([[5,1,5],[1,2,1]]).float()
weights_true = torch.transpose(weights_true,0,1)
bias_true = torch.tensor([1,2])
y_true = torch.mm(x**2,weights_true) + torch.mm(x,weights_true) + bias_true
print(f'x: {x.shape}, weights: {weights_true.shape}, bias: {bias_true.shape}, y: {y_true.shape}')
'''

x: torch.Size([400, 3]), weights: torch.Size([3, 2]), bias: torch.Size([1]), y: torch.Size([400, 2])


"\n# Generating data of 3 dim input and 2 dim output\nn, d = 500, 3\nx = torch.FloatTensor(n, d).uniform_(-1, 1)\nweights_true = torch.tensor([[5,1,5],[1,2,1]]).float()\nweights_true = torch.transpose(weights_true,0,1)\nbias_true = torch.tensor([1,2])\ny_true = torch.mm(x**2,weights_true) + torch.mm(x,weights_true) + bias_true\nprint(f'x: {x.shape}, weights: {weights_true.shape}, bias: {bias_true.shape}, y: {y_true.shape}')\n"

In [40]:
#ustom Model using nn.Linear modules
class Linear(nn.Module):
  def __init__(self, input_dim, num_hidden):
    super(Linear, self).__init__()
    self.init = torch.rand(input_dim, num_hidden).float()
    self.weights = torch.nn.Parameter(self.init, requires_grad=True)
    self.bias = torch.zeros(num_hidden,)
  
  def __call__(self, x):
    self.x = x
    return torch.mm(x, self.weights) + self.bias

In [41]:
# Torchmodel
class TorchModel(nn.Module):
  def __init__(self, input_dim, num_hidden):
    super(TorchModel, self).__init__()
    self.linear1 = Linear(input_dim, num_hidden)
    self.relu1 = nn.ReLU()
    self.linear2 = Linear(num_hidden, 12)
    self.relu2 = nn.ReLU()
    self.linear3 = Linear(12, 2)
  
  def forward(self, x):
    l1 = self.linear1(x)
    r1 = self.relu1(l1)
    l2 = self.linear2(r1)
    r2 = self.relu2(l2)
    l3 = self.linear3(r2)
    return l3

In [42]:
# Now we run the training loop
from typing import Callable
import pdb

def torch_fit(x, y, model: Callable, loss: Callable, lr: float, num_epochs: int):
  # pdb.set_trace()
  optimizer = torch.optim.SGD(model.parameters(), lr=lr)
  for epoch in range(num_epochs):
    optimizer.zero_grad() # Initialize gradient as zero
    y_pred_tensor = model(x) # Forward Pass
    loss_value = loss(y_pred_tensor, y) # Compute loss with MSE
    if epoch % 50 == 0:
      print(f'Epoch {epoch}, loss {loss_value}')
    loss_value.backward() # Use autogradient to compute backward pass
    optimizer.step() # Update weights 

loss = nn.MSELoss()

model = TorchModel(d, 22)
torch_fit(x, y_true, model=model, loss=loss, lr=0.0035, num_epochs=1000)


Epoch 0, loss 370.4049987792969
Epoch 50, loss 1.4712131023406982
Epoch 100, loss 0.5658929944038391
Epoch 150, loss 0.29855334758758545
Epoch 200, loss 0.22954867780208588
Epoch 250, loss 0.20309807360172272
Epoch 300, loss 0.18567590415477753
Epoch 350, loss 0.17206153273582458
Epoch 400, loss 0.1607278287410736
Epoch 450, loss 0.1512012630701065
Epoch 500, loss 0.14297333359718323
Epoch 550, loss 0.13591457903385162
Epoch 600, loss 0.12975789606571198
Epoch 650, loss 0.12450011074542999
Epoch 700, loss 0.12000495940446854
Epoch 750, loss 0.11597707122564316
Epoch 800, loss 0.11244658380746841
Epoch 850, loss 0.10939356684684753
Epoch 900, loss 0.10677307844161987
Epoch 950, loss 0.1044454500079155


In [43]:
def plot_intereactive_3d(x, y, y_pred=None):
  import plotly.graph_objects as go

  fig = go.Figure()
  fig.add_trace(go.Scatter3d(x = x[:,0],
                    y = x[:,1],
                    z = y.reshape([-1]),
                    opacity=0.5, mode='markers', name='Underlying Function'
                    ))
 
  if y_pred is not None:
    fig.add_trace(go.Scatter3d(x = x[:,0],
                   y = x[:,1],
                   z = y_pred.reshape([-1]),
                   opacity=0.5, mode='markers', name='Predicted Function'
                  ))
    
  fig.update_layout(scene = dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Y'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
  fig.show()

In [44]:
from sklearn.manifold import TSNE
X_red = TSNE(n_components=2).fit_transform(x)
y_true_red = TSNE(n_components=1).fit_transform(y_true)
y_pred_red = TSNE(n_components=1).fit_transform(model(x).detach())
print(f'X_red: {X_red.shape}, y_true_red: {y_true_red.shape}, y_pred_red: {y_pred_red.shape}')

plot_intereactive_3d(X_red,y_true_red,y_pred_red)

X_red: (400, 2), y_true_red: (400, 1), y_pred_red: (400, 1)
